In [5]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display 
from sklearn import metrics
import math
import pandas as pd
import numpy as np

In [89]:
df = pd.read_csv("train.csv",low_memory=False,parse_dates=["saledate"])

In [90]:
df.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h1>Kategorik değişkenleri numeric hale getirmek</h1>

In [91]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: object

In [92]:
for n,c in df.items():  #n kolonları c ise verileri ifade ediyor
    print(n)
    print("---")
    print(c)
    break

SalesID
---
0         1139246
1         1139248
2         1139249
3         1139251
4         1139253
           ...   
401120    6333336
401121    6333337
401122    6333338
401123    6333341
401124    6333342
Name: SalesID, Length: 401125, dtype: int64


In [93]:
from pandas.api.types import is_string_dtype
#string olan dataları numeric hale getireceğiz
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c): #sütunun string veri tipinde olup olmadığını kontrol eder. Eğer öyleyse, içeri girer. 
            df[n]=c.astype("category").cat.as_ordered() #sütunu kategorik bir veri tipine dönüştürür. İlk olarak, astype("category") metoduyla sütunu bir kategori tipine dönüştürürüz. Ardından, cat.as_ordered() metoduyla bu kategorileri sıralarız. Son olarak, bu sütunu veri çerçevesinde ilgili sütunla değiştiririz.

In [94]:
df['UsageBand'] = df['UsageBand'].astype('category')
#train sete uygulandıktan sonra aynı kategori değişimleri olsun diye validation ve traine bu uygulanır
def apply_cats(df,train):
    for n,c in df.items():
        if train[n].dtype =="category":
            df[n] = pd.Categorical(c, categories = train[n].cat.categories, ordered = True)

In [95]:
train_cats(df)

In [96]:
df["UsageBand"]

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): ['High' < 'Low' < 'Medium']

<h1>Bilgisayara High,Medium,Low sıralamasını vermemiz lazım</h1>

In [97]:
df["UsageBand"] = df["UsageBand"].cat.set_categories(["High", "Medium", "Low"], ordered=True)


In [98]:
df["UsageBand"].cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

In [99]:
df["UsageBand"].cat.codes #Dönüşümü yukarıda yaptık ama dönüşüm sadece arka plana işledi. Onlara böyle eriştik.

0         2
1         2
2         0
3         0
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

<h1>-1'ler NaN değerleri işaret ediyor. Verinin daha güzel durması için onları 0a çekip değerleri değiştirelim. Zaten değişiklikler önplanda oldu sadece</h1>


In [100]:
from pandas.api.types import is_numeric_dtype
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes + 1

In [101]:
numericalize(df, df["UsageBand"], "UsageBand")

In [102]:
df["UsageBand"]

0         3
1         3
2         1
3         1
4         2
         ..
401120    0
401121    0
401122    0
401123    0
401124    0
Name: UsageBand, Length: 401125, dtype: int8

<h1>Böylece kategorik değişlenleri numeric hale getirdik</h1>

<h2>Datetime kolonu</h2>

In [103]:
df["saledate"].dt.year

0         2006
1         2004
2         2004
3         2011
4         2009
          ... 
401120    2011
401121    2011
401122    2011
401123    2011
401124    2011
Name: saledate, Length: 401125, dtype: int32

<h2>Attribute eklemek</h2>

In [108]:
def add_datepart(df, dt_name, drop=True, time=False):
    dt_column = df[dt_name]
    column_type = dt_column.dtype

    attr = ['year','month','dayofweek','dayofyear',
            'is_month_end','is_month_start','is_quarter_end','is_quarter_start','is_year_end','is_year_start']

    for a in attr:
        df["Date" + a.capitalize()] = getattr(dt_column.dt, a)
        df["Date" + "Elapsed"] = dt_column.astype(np.int64) // 10 ** 9

    if drop:
        df.drop(dt_name,axis=1,inplace=True)

In [109]:
add_datepart(df,"saledate")

In [110]:
df

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,DateElapsed,DateMonth,DateDayofweek,DateDayofyear,DateIs_month_end,DateIs_month_start,DateIs_quarter_end,DateIs_quarter_start,DateIs_year_end,DateIs_year_start
0,1139246,66000,999089,3157,121,3.0,2004,68.0,3,521D,...,1163635200,11,3,320,False,False,False,False,False,False
1,1139248,57000,117657,77,121,3.0,1996,4640.0,3,950FII,...,1080259200,3,4,86,False,False,False,False,False,False
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,1,226,...,1077753600,2,3,57,False,False,False,False,False,False
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,1,PC120-6E,...,1305763200,5,3,139,False,False,False,False,False,False
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,2,S175,...,1248307200,7,3,204,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401120,6333336,10500,1840702,21439,149,1.0,2005,NaN,0,35NX2,...,1320192000,11,2,306,False,False,False,False,False,False
401121,6333337,11000,1830472,21439,149,1.0,2005,NaN,0,35NX2,...,1320192000,11,2,306,False,False,False,False,False,False
401122,6333338,11500,1887659,21439,149,1.0,2005,NaN,0,35NX2,...,1320192000,11,2,306,False,False,False,False,False,False
401123,6333341,9000,1903570,21435,149,2.0,2005,NaN,0,30NX,...,1319500800,10,1,298,False,False,False,False,False,False


<h1>Mevcut datasetini kaydetmek için Feather Format</h1>

In [111]:
import os

In [114]:
os.makedirs("tmp",exist_ok=True) #klasör zaten varsa hata vermesin diye
df.to_feather("tmp/Bulldozers_1")